In [ ]:
from typing import List, Dict, Any, Optional
from langgraph.graph import StateGraph, END

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI
import pandas as pd
import numpy as np
import yfinance as yf  
import os
from langgraph.prebuilt import ToolNode 
from langchain.agents.agent_types import AgentType
import os
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings, GoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
config = dotenv_values("./keys/.env")
with open("./keys/complete-tube-421007-208a4862c992.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key
OPENAI_API_KEY = config.get("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
# Ensure you have your OpenAI API key set as an environment variable
if "OPENAI_API_KEY" not in os.environ:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [ ]:
period=20

In [ ]:
data = yf.download("AAPL", period=f"{period+50}d")

In [ ]:
data

In [ ]:
import yfinance as yf
import pandas as pd

@tool
def calculate_bollinger_z_score(symbol:str, period: int = 20)-> str:
    """This tool is used to calculate the Bollinger Z Score
    I get A symbol name in the variable symbol and a period and I calculate a the Score

    Args:
        symbol (str): symbol to calculate the z-score.
        period  (int): period to calculate the score
        Returns:
        str:  Message with the relevant valuest

    """
    # Fetch historical data
    data = yf.download(symbol, period=f"{period+50}d")
    if data.empty:
        return f"Could not find data for symbol: {symbol}"

    closes = data['Close']
    rolling_mean = closes.rolling(window=period).mean()
    rolling_std = closes.rolling(window=period).std()
    upper_band = rolling_mean + (2 * rolling_std)
    lower_band = rolling_mean - (2 * rolling_std)
    z_score = (closes - rolling_mean) / rolling_std

    latest_z_score = z_score.iloc[-1].values[0]
    latest_upper_band = upper_band.iloc[-1].values[0]
    latest_lower_band = lower_band.iloc[-1].values[0]
    latest_close = closes.iloc[-1].values[0]
    
    message = f"""
    Symbol: {symbol}, Period: {period}\n
    Latest Close: {latest_close:.2f}\n
    Latest Upper Band: {latest_upper_band:.2f}\n
    Latest Lower Band: {latest_lower_band:.2f}\n
    Latest Bollinger Z-score: {latest_z_score:.2f}
    """
    return message


In [ ]:
calculate_bollinger_z_score("UBS")

In [ ]:
#z_score.iloc[-1].values[0]

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
# Set up the model

from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
tools = [calculate_bollinger_z_score]

llm_with_tools = llm.bind_tools(tools)

In [ ]:

import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.messages import ToolMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


def chatbot(state: State):
    """Always use tools to fulfill user requests.
    You are a clever chatbot which recomemd what to do with the stock user request based in the interpretation of the Bolling z-score

    the Bollinger Z-Score is a technical indicator used to assess the position of a security's price relative to its Bollinger Bands. It's a normalized measure that helps traders understand whether a price is relatively high or low compared to its recent trading range.

    Here's how to interpret the Bollinger Z-Score:
    
    Z-Score of 0: The current price is exactly at the middle Bollinger Band (the simple moving average).
    Positive Z-Score: The price is above the middle Bollinger Band. The higher the Z-Score, the further the price is above the average and the more "overbought" the security may be considered.
    Negative Z-Score: The price is below the middle Bollinger Band. The lower the Z-Score, the further the price is below the average and the more "oversold" the security may be considered.
    General guidelines for interpretation:
    
    Z-Score > +2: The price is considered overbought. It might be a good time to consider selling or shorting the security.
    Z-Score < -2: The price is considered oversold. It might be a good time to consider buying the security.
    Z-Score between -2 and +2: The price is within the "normal" trading range.
    Important considerations:
    
    Trend: The Z-Score should be interpreted in conjunction with the overall trend. In an uptrend, overbought conditions may persist, while in a downtrend, oversold conditions may persist.
    Volatility: The Z-Score is affected by volatility. A more volatile security will have a wider Bollinger Band and potentially larger Z-Score fluctuations.
    False Signals: The Z-Score can generate false signals, especially in trending markets. It's important to use other technical indicators and fundamental analysis to confirm trading decisions.
    
    1. If you do not have enough inputs to execute a tool then you can ask for more information.
    2. calculate the Score and return am answer based in what the tools return you and following the guidelines to  Interpretate it I give you 
    """
    # Filter out messages with image type
    # text_messages = [msg for msg in state["messages"] if msg['content'][0].get("type") != "image"]
    text_messages = [
        msg for msg in state["messages"]
        if not (isinstance(msg.content, list) and msg.content[0].get("type") == "image_url")
    ]

    # Invoke LLM with only text messages
    return {"messages": [llm_with_tools.invoke(text_messages)]}


graph_builder.add_node("chatbot", chatbot)

In [ ]:

from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

memory = MemorySaver()

tool_node = ToolNode(tools)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
    {"tools": "tools", "__end__": "__end__"},
)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
from langchain_core.messages import HumanMessage
import pprint
config = {"configurable": {"thread_id": "20"}}
graph.invoke(
        {
            "messages": [
                (
                    "user",
                    "Recomend if buy or sell UBS based in Calculations of the Bollinger Z-score  within a 30-day period.",
                )
            ]
        },
        config,
    )["messages"][-1].content

     


In [ ]:
graph.invoke(
        {
            "messages": [
                (
                    "user",
                    "Recomend if buy or sell TSLA based in Calculations of the Bollinger Z-score  within a 20-day period.",
                )
            ]
        },
        config,
    )["messages"][-1].content